# Prepare labels dictionary

This notebooks will help you to download the trained model, display cluster reports and label the clusters in an intercative way.

In [ ]:
%load_ext dotenv
%dotenv
%matplotlib inline

import os
from os.path import join
import sys

sys.path.append("../")

import argparse
import tempfile
import pickle

import matplotlib.pyplot as plt

from azureml.core.model import Model
from azureml.core import Workspace

from mlops.common.workspace import get_workspace
from ml.utils.image import read_images_from_folder

In [ ]:
# Restore AML workspace from config.json file (can be downloaded through the portal)
ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')


# Download your model
model_name = "YOUR_MODEL_NAME"
models_dir = "../models"

model = Model(ws, model_name, version=None)
model.download(models_dir)

In [ ]:
# Label representatives of each cluster
labels = {}

report_folder = os.path.join(models_dir, "outputs", "report")
clusters_images, clusters_names = read_images_from_folder(report_folder)

for cluster_img, cluster_name in zip(clusters_images, clusters_names):
    plt.imshow(cluster_img)
    plt.show()
    raw_labels = input("Please enter cluster labels (comma-separated)")
    cluster_id = int(cluster_name.split('_')[1])
    labels_parsed = raw_labels.split(',')
    if cluster_id == -1:
        labels_parsed.append("noise points")
    labels[cluster_id] = labels_parsed
    
print(labels)
pickle.dump(labels, open('labels.pickle', 'wb'))